In [24]:
import wrds
db=wrds.Connection(wrds_username='hlanfran')
import numpy as np
import pandas as pd

Loading library list...
Done


In [145]:
Rf=db.raw_sql("select  mcaldt,tmytm "
           "from crsp.tfz_mth_rf "           
            "where kytreasnox = 2000001 "
           "and mcaldt>='1990-01-01'"
            "and mcaldt<='2021-12-31'", date_cols=['mcaldt'])

#transform annualized and continuously compounded returns into simple and monthly returns:
Rf['tmytm']=np.exp(Rf['tmytm']/12/100)-1
Rf=Rf.rename(columns={ "mcaldt": "date","tmytm": "rf"})

#use own index instead
# get index returns
data_index=db.raw_sql("select  date,vwretd "
           "from crsp.msi "
           "where date>='1900-01-01'"
            "and date<='2021-12-31'", date_cols=['date'])


# get stock returns
crsp_m = db.raw_sql("""
                      select a.permno, a.date, 
                      b.shrcd, b.exchcd,
                      a.ret, a.shrout, a.prc, a.hsiccd 
                      from crsp.msf as a
                      left join crsp.msenames as b
                      on a.permno=b.permno
                      and b.namedt<=a.date
                      and a.date<=b.nameendt
                      where a.date between '01/01/1990' and '12/31/2021'
                      and b.exchcd between 1 and 2
                      and b.shrcd between 10 and 11
                      """, date_cols=['date']) 
msf=crsp_m.drop(['shrcd','exchcd'],axis=1)
del(crsp_m) 


In [146]:

#variables
msf['permno'] = msf['permno'].astype(int)
msf['size'] = msf['shrout'] * msf['prc'].abs()
msf['const'] = 1
msf.sort_values(['permno','date'])

msf['size_lag']= msf.groupby('permno')['size'].shift(1).to_frame()

obs=msf[['const','permno']].groupby(['permno']).sum().reset_index().rename(columns={'const': 'obs'})
msf = pd.merge(msf, obs, how='left', left_on=['permno'],right_on=['permno'])



#################################
# Compute excess returns
#################################
data_index = pd.merge(data_index, Rf, how='left', left_on=['date'],right_on='date')
data_index['mprem']=(data_index['vwretd']-data_index['rf'])
data = pd.merge(msf, data_index, how='left', left_on=['date'],right_on='date')
data['exret']=data['ret']-data['rf']

In [147]:
print('duplicates in data on returns?')
print(data.duplicated(subset=['date','permno']).sum(),'\n')

print(' How many obervations do we have in a month?')
print('max:', max(data[['date','const']].groupby('date').sum()['const'].values))
print('min:', min(data[['date','const']].groupby('date').sum()['const'].values))

print('\n How often do we observe a given stock?')
print('max:', max(data['obs'].values))
print('min:', min(data['obs'].values))

print('\n Let us limit our attention to stocks we observe often enough...')


duplicates in data on returns?
0 

 How many obervations do we have in a month?
max: 2462
min: 1370

 How often do we observe a given stock?
max: 384
min: 1

 Let us limit our attention to stocks we observe often enough...


In [148]:
#Keep interesting data
data=data[data.obs==384]

In [149]:
#Get 1995 year
data_filter = data[(data['date'] > '1995-01-01') & (data['date'] < '1995-12-30')]

In [150]:
#Sort unique stocks
data_unique = data_filter.drop_duplicates(subset=['permno'])

In [151]:
data_unique["market-capitalization"] = data_unique["shrout"] * data_unique["prc"]

C:\Users\hugol\AppData\Local\Temp\ipykernel_17984\4005763058.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_unique["market-capitalization"] = data_unique["shrout"] * data_unique["prc"]


In [152]:
#get largest stocks
data_unique = data_unique.nlargest(100,"market-capitalization")

In [133]:
#filter data to keep only largest stocks
data = data[data["permno"].isin(data_unique["permno"].values)]

In [155]:
#assign industry
data["industry"] = data["hsiccd"].map(lambda x: get_industry(x))

In [158]:
#get month number
data["month"] = data["date"].map(lambda x: x.month)

In [159]:
data

,permno,date,ret,shrout,prc,hsiccd,size,const,size_lag,obs,vwretd,rf,mprem,exret,industry,month
6855,10516,1990-01-31,-0.134054,281065.0,20.000000,2046.0,5.621300e+06,1,NaN,384,-0.070114,0.006390,-0.076503,-0.140444,Consumer Nondurables,1
6856,10516,1990-02-28,0.050000,281065.0,21.000000,2046.0,5.902365e+06,1,5.621300e+06,384,0.014900,0.006441,0.008459,0.043559,Consumer Nondurables,2
6857,10516,1990-03-30,0.083333,283801.0,22.750000,2046.0,6.456473e+06,1,5.902365e+06,384,0.024148,0.006688,0.017460,0.076645,Consumer Nondurables,3
6858,10516,1990-04-30,0.034066,283801.0,23.500000,2046.0,6.669324e+06,1,6.456473e+06,384,-0.028283,0.006283,-0.034567,0.027782,Consumer Nondurables,4
6859,10516,1990-05-31,0.127660,283801.0,26.500000,2046.0,7.520726e+06,1,6.669324e+06,384,0.088935,0.006132,0.082803,0.121528,Consumer Nondurables,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602802,85914,2021-08-31,0.037027,245964.0,116.510002,5731.0,2.865727e+07,1,2.814064e+07,384,0.027147,0.000034,0.027113,0.036993,"Wholesale, Retail, and Some Services (Laundrie...",8
602803,85914,2021-09-30,-0.086688,245964.0,105.709999,5731.0,2.600085e+07,1,2.865727e+07,384,-0.042243,0.000038,-0.042281,-0.086726,"Wholesale, Retail, and Some Services (Laundrie...",9
602804,85914,2021-10-29,0.156371,245964.0,122.239998,5731.0,3.006664e+07,1,2.600085e+07,384,0.064657,0.000042,0.064614,0.156329,"Wholesale, Retail, and Some Services (Laundrie...",10
602805,85914,2021-11-30,-0.125818,245964.0,106.860001,5731.0,2.628371e+07,1,3.006664e+07,384,-0.018347,0.000068,-0.018414,-0.125886,"Wholesale, Retail, and Some Services (Laundrie...",11


In [160]:
industry_portofolios = pd.read_csv("12_Industry_Portfolios.CSV")

In [161]:
industry_portofolios

,Date,NoDur,Durbl,Manuf,Enrgy,Chems,BusEq,Telcm,Utils,Shops,Hlth,Money,Other
0,192607,1.45,15.55,3.67,-1.18,8.01,3.16,0.83,7.04,0.11,1.77,-0.02,2.22
1,192608,3.97,3.68,2.42,3.47,5.14,1.97,2.17,-1.69,-0.71,4.25,4.47,4.34
2,192609,1.14,4.80,-0.07,-3.39,5.30,-0.34,2.41,2.04,0.21,0.69,-1.61,0.37
3,192610,-1.24,-8.23,-3.16,-0.78,-4.55,-5.38,-0.11,-2.63,-2.29,-0.57,-5.51,-2.73
4,192611,5.20,-0.19,3.82,0.01,5.11,4.79,1.63,3.71,6.43,5.42,2.34,2.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1144,202111,-3.77,1.70,-1.09,-5.46,0.43,0.99,-7.24,-1.96,1.50,-4.18,-5.67,-3.97
1145,202112,8.05,-4.44,4.15,3.09,9.60,1.78,3.86,8.57,1.34,6.74,4.85,5.30
1146,202201,-0.67,-10.15,-5.57,17.35,-6.25,-8.27,-1.99,-2.10,-8.95,-8.60,-0.62,-6.21
1147,202202,-0.53,-7.21,-0.16,8.08,-3.47,-5.14,0.26,-1.11,-1.55,-1.05,-1.66,0.22


In [162]:
192607/12


16050.583333333334

### Helpers

In [54]:
def get_industry(number):

    if(((number>=100) and (number<=999)) or (number>=2000 and number<=2399) or  (number>=2700 and number<=2749) or  (number>=2770 and number<=2799) or  (number>=3100 and number<=3199)  or  (number>=3940 and number<=3989)):
        return  "Consumer Nondurables"

    if((number>=2500 and number<=2519) or (number>=2590 and number<=2599) or (number>=3630 and number<=3659) or (number>=3710 and number<=3711) or (number>=3714 and number<=3714) or (number>=3716 and number<=3716) or (number>=3750 and number<=3751) or (number>=3792 and number<=3792) or (number>=3900 and number<=3939) or  (number>=3900 and number<=3939)):
        return "Consumer Durables"
    
    if((number>=2520 and number<=2589) or  (number>=2600 and number<=2699) or  (number>=2750 and number<=2769) or  (number>=3000 and number<=3099) or (number>=3200 and number<=3569) or (number>=3580 and number<=3629) or (number>=3700 and number<=3709) or(number>=3712 and number<=3713) or(number>=3715 and number<=3715) or(number>=3717 and number<=3749) or(number>=3752 and number<=3791) or(number>=3793 and number<=3799) or(number>=3830 and number<=3839) or(number>=3860 and number<=3899)):
        return "Manufacturing"
    
    if((number>=1200 and number<=1399) or  (number>=2900 and number<=2999)):
        return "Oil, Gas, and Coal Extraction and Products"
    
    if((number>=2800 and number<=2829) or  (number>=2840 and number<=2899)):
        return "Chemicals and Allied Products"
    
    if((number>=3570 and number<=3579) or  (number>=3660 and number<=3692) or (number>=3694 and number<=3699) or  (number>=3810 and number<=3829) or  (number>=7370 and number<=7379)):
        return "Business Equipment -- Computers, Software, and Electronic Equipment"
    
    if((number>=4800 and number<=4899)):
        return "Telephone and Television Transmission"
    
    if((number>=4900 and number<=4949)):
        return "Utilities"
    
    if((number>=5000 and number<=5999) or  (number>=7200 and number<=7299) or (number>=7600 and number<=7699)):
        return "Wholesale, Retail, and Some Services (Laundries, Repair Shops)"
    
    if((number>=2830 and number<=2839) or  (number>=3693 and number<=3693) or (number>=3840 and number<=3859)  or (number>=8000 and number<=8099)):
        return "Healthcare, Medical Equipment, and Drugs"
    
    if((number>=6000 and number<=6999)):
        return "Finance"
    
    else:
        return "Other -- Mines, Constr, BldMt, Trans, Hotels, Bus Serv, Entertainment"

In [ ]:
 1 NoDur  Consumer Nondurables -- Food, Tobacco, Textiles, Apparel, Leather, Toys
          0100-0999
          2000-2399
          2700-2749
          2770-2799
          3100-3199
          3940-3989

 2 Durbl  Consumer Durables -- Cars, TVs, Furniture, Household Appliances
          2500-2519
          2590-2599
          3630-3659
          3710-3711
          3714-3714
          3716-3716
          3750-3751
          3792-3792
          3900-3939
          3990-3999

 3 Manuf  Manufacturing -- Machinery, Trucks, Planes, Off Furn, Paper, Com Printing
          2520-2589
          2600-2699
          2750-2769
          3000-3099
          3200-3569
          3580-3629
          3700-3709
          3712-3713
          3715-3715
          3717-3749
          3752-3791
          3793-3799
          3830-3839
          3860-3899

 4 Enrgy  Oil, Gas, and Coal Extraction and Products
          1200-1399
          2900-2999

 5 Chems  Chemicals and Allied Products
          2800-2829
          2840-2899

 6 BusEq  Business Equipment -- Computers, Software, and Electronic Equipment
          3570-3579
          3660-3692
          3694-3699
          3810-3829
          7370-7379

 7 Telcm  Telephone and Television Transmission
          4800-4899

 8 Utils  Utilities
          4900-4949

 9 Shops  Wholesale, Retail, and Some Services (Laundries, Repair Shops)
          5000-5999
          7200-7299
          7600-7699

10 Hlth   Healthcare, Medical Equipment, and Drugs
          2830-2839
          3693-3693
          3840-3859
          8000-8099

11 Money  Finance
          6000-6999

12 Other  Other -- Mines, Constr, BldMt, Trans, Hotels, Bus Serv, Entertainment